# Extract coordinates and generate movie.xyz

The next code generate a movie.xyz file from the output information of NWChem program, it search through the output file, in this case "salida.out" for the coordinates, it generates a dataframe that is stored in pkl format, the it reads from the dataframe file to generate the movie.xyz

The coordinates in the NWChem output are in atomic units to convert them to angstroms we multiply by 0.5291771057875306 ~ 1/1.889725989 

In [ ]:
import numpy as np
import pandas as pd

path = "salida.out"
N = 30
au2aFactor = 0.5291771057875306

dic = {}
dic["ida"] = {}
dic["atom"] = {}
dic["x"] = {}
dic["y"] = {}
dic["z"] = {}
dic["fx"] = {}
dic["fy"] = {}
dic["fz"] = {}
dic["frame"] = {}


ij = 0
f = open(path, "r")
idx = 0
for line in f:
    if "DFT ENERGY GRADIENTS" in line:
        line = next(f)
        line = next(f)
        line = next(f)
        for i in range(N):
            line = next(f)
            sp = line.split()
            
            ida = sp[0]
            atom = sp[1]
            x = sp[2]
            y = sp[3]
            z = sp[4]
            fx = sp[5]
            fy = sp[6]
            fz = sp[7]
            frame = ij
            
            dic["ida"][idx] = ida
            dic["atom"][idx] = atom
            dic["x"][idx] = float(x)*au2aFactor
            dic["y"][idx] = float(y)*au2aFactor
            dic["z"][idx] = float(z)*au2aFactor
            dic["fx"][idx] = float(fx)
            dic["fy"][idx] = float(fy)
            dic["fz"][idx] = float(fz)        
            dic["frame"][idx] = int(frame)
            idx+=1
        
        ij+=1

f.close()

path_out = "aot-jqkk.pkl"
df = pd.DataFrame(dic)
df.to_pickle(path_out)

path_info = "aot-jqkk.pkl"
df = pd.read_pickle(path_info)

path_movie = "movie_from_min.xyz"
f = open(path_movie, "w")

frames = df["frame"].unique()
N = len(df[df["frame"] == 0])
for frame in frames:
    
    tmp_df = df[df["frame"] == frame]
    f.write(f"{len(tmp_df)}")
    f.write(f"\n  {frame}\n")
    for idx in tmp_df.index:
        
        atom = tmp_df["atom"].loc[idx]
        x = tmp_df["x"].loc[idx]
        y = tmp_df["y"].loc[idx]
        z = tmp_df["z"].loc[idx]
        
        f.write(f"{atom}")
        f.write(f"{x:10.4f}{y:10.4f}{z:10.4f}\n")
        
    
    
f.close()